In [1]:
import pandas as pd
import os
import branca
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import folium as f
import geojson
from folium.features import GeoJsonPopup as pp
import mapclassify as mc

wgs84 = {'init':'EPSG:4326'}
bng = {'init':'EPSG:27700'}

In [2]:
# get the pharmacies in
fl = './../../data/food hazard/lines.geojson'
lines = gpd.read_file(fl)
#lines2 = lines.to_json()

In [3]:
lines = lines.rename(columns={'Country.of.Origin': 'Country of Origin', 'Notified.by':'Notified by','number.of.links':'Number of Food Hazards identified'  })

In [4]:
fl = './../../data/food hazard/safety.csv'
safety  = pd.read_csv(fl, index_col=None)

In [5]:
# get the pharmacies in
fl = './../../data/food hazard/points.geojson'
points = gpd.read_file(fl)
points = points.merge(safety, on = 'name', how = 'left')

In [6]:
points = points.sort_values(['degree_centrality'], axis=0, ascending=False, inplace=False, kind='quicksort')
points['rank'] = np.arange(1, 151, 1)
points = points.sort_values(['name'], axis=0, ascending=True, inplace=False, kind='quicksort')
points = points.rename(columns={'name':'Name', 'degree_centrality': 'Country importance', 'Rank':'Food Safety and Quality Rank (1 is the best)', 'rank': 'Country Importance Rank (1 is the best)'})
points

,Name,x,y,degree,Country importance,out_degree,in_degree,geometry,Food Safety and Quality Rank (1 is the best),Score,difference from previous year,Country Importance Rank (1 is the best)
0,Afghanistan,66.004734,33.835231,6,0.040268,0.040268,0.040268,POINT (66.00473 33.83523),NaN,NaN,NaN,107
2,Albania,20.049834,41.142450,20,0.134228,0.134228,0.134228,POINT (20.04983 41.14245),NaN,NaN,NaN,83
35,Algeria,2.617323,28.158938,18,0.120805,0.120805,0.120805,POINT (2.61732 28.15894),74.0,53.0,-4.6,85
3,Andorra,1.560544,42.542291,1,0.006711,0.006711,0.006711,POINT (1.56054 42.54229),NaN,NaN,NaN,139
1,Angola,17.537368,-12.293361,1,0.006711,0.006711,0.006711,POINT (17.53737 -12.29336),92.0,44.9,1.9,148
...,...,...,...,...,...,...,...,...,...,...,...,...
140,Uzbekistan,63.140015,41.755542,35,0.234899,0.234899,0.234899,POINT (63.14002 41.75554),73.0,53.4,0.7,75
141,Venezuela,-66.181841,7.124224,55,0.369128,0.369128,0.369128,POINT (-66.18184 7.12422),47.0,66.9,-0.1,59
142,Vietnam,106.299147,16.646017,440,2.953020,2.953020,2.953020,POINT (106.29915 16.64602),77.0,51.7,0.3,19
143,Yemen,47.586762,15.909280,1,0.006711,0.006711,0.006711,POINT (47.58676 15.90928),109.0,30.2,-0.9,142


In [7]:
points2 = points.to_json()

In [8]:
lines2 = lines.to_json()

In [9]:
def ordinal(n):
    if n == 1:
        return "st"
    elif n == 2:
        return "nd"
    elif n == 3:
        return "rd"
    else: 
        return "th"

In [10]:
ordinal(5)

'th'

m=f.Map(location=[50, 2], zoom_start=3 , tiles='cartodbpositron')

f.TileLayer('CartoDB dark_matter').add_to(m)

colordict = {0:'#374545' , 1: '#2f5565' , 2: '#24749f', 3: '#1892d9', 4: '#1892d9'}

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
style_function= lambda feature: {
                'opacity': 2,
                'weight': 0.2,
                'color': "#2F4F4F"
            }
f.features.GeoJson(lines, 
                   control = True, name = 'Food Hazards',
                  style_function=style_function,
                  highlight_function=highlight_function).add_to(m)




    
f.LayerControl().add_to(m)

#f.Choropleth(
#    lines[lines.geometry.length>0.001],
#    line_weight=1,
#   line_color='blue',
#    style_function=style_function
#).add_to(m)
 

    
    
#f.GeoJson(lines,name='lines').add_to(m)



for x, y, name, degree, in_degree, out_degree, degree_quartile, rank, Rank in zip(points['x'], points['y'], points['name'], points['degree'], points['in_degree'],points['out_degree'], points['degree_quartile'], points['rank'], points['Rank']):
    f.CircleMarker(
        [y, x],
        radius=(degree/100)+10,
        popup = #f"Route {rank} to {ordinal(rank)}",
        f.Popup(f"<b>{str(name)}</b> <br> <br> Ranked <b>{str(rank)}</b>{str(ordinal(rank))} important identifier and <br> <b>{str(round(Rank,0))}</b>{str(ordinal(Rank))} in food Safety and Quality" , max_width = 900),
        color='b',
        key_on = degree_quartile,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[degree_quartile],
        fill=True,
        fill_opacity=0.8, control = True, name = 'Countries'
        ).add_to(m)

m

In [11]:
m.save('index.html')

NameError: name 'm' is not defined

In [14]:
# Load an empty map
from keplergl import KeplerGl
map_1 = KeplerGl(height=800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [15]:
map_1.add_data(data=points2, name='Countries')


In [16]:
map_1.add_data(data=lines2, name='Food Hazards')

In [17]:
map_1

KeplerGl(data={'Countries': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "proper…

In [20]:
map_1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '89okhlf', 'type': …

In [21]:
map_1.save_to_html(file_name='index2.html', read_only=True)

Map saved to index2.html!
